In [1]:
import pandas as pd
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import sklearn as sk
import requests
import time
from sklearn.model_selection import train_test_split
from sklearn import linear_model 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler

In [2]:
#데이터 가져오기
df = pd.read_csv("raw_data/df_grouped_rate.csv", encoding ='utf-8')
cgoongu = pd.read_csv("raw_data/cgoongoo1.csv", encoding ='utf-8')

In [3]:
df_merged = pd.merge(df, cgoongu, on=['district'])
df_merged.drop(['district'], axis=1, inplace =True)

In [4]:
df_merged.columns

Index(['year', 'code', 'sales_total', 's_store_no_of_store',
       's_store_no_of_opening', 's_store_no_of_closing', 's_work_female_40s',
       's_work_female_50s', 's_work_female_60s', 'b_facil_total',
       's_float_male', 's_float_female', 'b_apt_avg_price',
       'b_income_avg_monthly_inc', 's_work_female', 's_work_male',
       'sales_weekday', 'sales_female', 'sales_2030s', 'sales_06_11',
       'sales_11_14', 'sales_14_17', 'sales_17_21', 'sales_21_24', 'cgoongoo'],
      dtype='object')

In [5]:
df_merged

,year,code,sales_total,s_store_no_of_store,s_store_no_of_opening,s_store_no_of_closing,s_work_female_40s,s_work_female_50s,s_work_female_60s,b_facil_total,...,s_work_male,sales_weekday,sales_female,sales_2030s,sales_06_11,sales_11_14,sales_14_17,sales_17_21,sales_21_24,cgoongoo
0,2018,CS100001,2.192504e+09,34,4,3,99.0,78.0,17.0,204,...,356.0,0.713984,0.347060,0.332434,0.009393,0.403097,0.152406,0.365022,0.070083,11110
1,2018,CS100002,1.456354e+08,1,0,1,99.0,78.0,17.0,204,...,356.0,0.759324,0.444114,0.462322,0.000000,0.492581,0.174083,0.323758,0.009578,11110
2,2018,CS100003,1.382572e+08,7,1,0,99.0,78.0,17.0,204,...,356.0,0.802200,0.366246,0.446717,0.000000,0.528620,0.121236,0.312228,0.037916,11110
3,2018,CS100004,5.676134e+08,10,1,2,99.0,78.0,17.0,204,...,356.0,0.578351,0.486286,0.593523,0.008071,0.298279,0.213005,0.456886,0.022930,11110
4,2018,CS100005,4.090006e+08,9,2,0,99.0,78.0,17.0,204,...,356.0,0.833051,0.482728,0.412484,0.005647,0.570256,0.206137,0.198317,0.019644,11110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43698,2019,CS300012,1.705865e+07,0,0,0,58.0,35.0,11.0,115,...,100.0,0.846890,0.234450,0.047847,0.000000,0.076555,0.741627,0.181818,0.000000,11740
43699,2019,CS300013,3.253261e+07,1,0,0,58.0,35.0,11.0,115,...,100.0,0.677729,0.641224,0.040929,0.045723,0.264749,0.324484,0.365044,0.000000,11740
43700,2019,CS300014,1.556122e+06,2,0,0,58.0,35.0,11.0,115,...,100.0,0.770492,0.678688,0.773770,0.000000,0.000000,0.000000,0.340984,0.596721,11740
43701,2019,CS300016,7.118222e+07,3,0,0,58.0,35.0,11.0,115,...,100.0,0.932742,0.158067,0.025457,0.051769,0.027485,0.756572,0.162709,0.001466,11740


In [19]:
scale_columns

['sales_total',
 's_store_no_of_store',
 's_store_no_of_opening',
 's_store_no_of_closing',
 's_work_female_40s',
 's_work_female_50s',
 's_work_female_60s',
 'b_facil_total',
 's_float_male',
 's_float_female',
 'b_apt_avg_price',
 'b_income_avg_monthly_inc',
 's_work_female',
 's_work_male',
 'sales_weekday',
 'sales_female',
 'sales_2030s',
 'sales_06_11',
 'sales_11_14',
 'sales_14_17',
 'sales_17_21',
 'sales_21_24',
 'cgoongoo']

In [22]:
#로버스트 스케일링
from sklearn.preprocessing import RobustScaler
scale_columns = [i for i in df_merged.columns if i not in ['year', 'cgoongoo', 'code',]]

rb = RobustScaler()
rb.fit(df_merged[scale_columns])
df_robust_scaled = rb.transform(df_merged[scale_columns])
df_robust_scaled = pd.DataFrame(df_robust_scaled, columns=scale_columns)
df_robust_scaled2= pd.concat([df_merged[['cgoongoo','code']],df_robust_scaled], axis=1)
df_robust_scaled.columns
model= sm.OLS.from_formula("sales_total ~ C(cgoongoo) +C(code) +"+" + ".join(scale_columns[1:]), data=df_robust_scaled2)
result =  model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:            sales_total   R-squared:                       0.426
Model:                            OLS   Adj. R-squared:                  0.425
Method:                 Least Squares   F-statistic:                     942.0
Date:                Thu, 26 Dec 2019   Prob (F-statistic):               0.00
Time:                        17:50:29   Log-Likelihood:            -2.2710e+05
No. Observations:              113265   AIC:                         4.544e+05
Df Residuals:                  113175   BIC:                         4.553e+05
Df Model:                          89                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [ ]:
"sales_total ~ C(cgoongoo) +C(code) +"+" + ".join(scale_columns[1:]